In [49]:
import time
import pandas as pd
import numpy as np
from google.colab import drive
import os

# YOU MUST RUN THIS FOLLOWING LINE THE FIRST TIME YOU RUN THE FILE
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load and Format Data

## CRSP/COMPUSTAT Merged

In [50]:
# Load Data
CRSP_COMPUSTAT_MERGED = pd.read_csv('/content/drive/Shared drives/Financial Modeling and Testing/Project 1/Data/Original/CRSP_COMPUSTAT_MERGED.csv')
CRSP_COMPUSTAT_MERGED.head()

,GVKEY,LPERMNO,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,conm,curcdq,datacqtr,datafqtr,atq,ceqq,cheq,ltq,niq,exchg,costat,dlrsn,sic,dldte
0,1000,25881,12/31/1970,1970,$4.000,INDL,C,D,STD,A & E PLASTIK PAK INC,USD,1970Q4,1970Q4,$nan,$10.544,$nan,$nan,$-1.064,12,I,$9.000,3089,06/30/1978
1,1000,25881,03/31/1971,1971,$1.000,INDL,C,D,STD,A & E PLASTIK PAK INC,USD,1971Q1,1971Q1,$nan,$nan,$nan,$nan,$0.346,12,I,$9.000,3089,06/30/1978
2,1000,25881,06/30/1971,1971,$2.000,INDL,C,D,STD,A & E PLASTIK PAK INC,USD,1971Q2,1971Q2,$nan,$nan,$nan,$nan,$0.152,12,I,$9.000,3089,06/30/1978
3,1000,25881,09/30/1971,1971,$3.000,INDL,C,D,STD,A & E PLASTIK PAK INC,USD,1971Q3,1971Q3,$nan,$nan,$nan,$nan,$-0.672,12,I,$9.000,3089,06/30/1978
4,1000,25881,12/31/1971,1971,$4.000,INDL,C,D,STD,A & E PLASTIK PAK INC,USD,1971Q4,1971Q4,$29.330,$8.381,$2.557,$20.949,$-2.144,12,I,$9.000,3089,06/30/1978


### Filtering

In [51]:
# Filter out 6000-6999 Range Companies and make sure traded on American Exchange
CRSP_COMPUSTAT_MERGED = CRSP_COMPUSTAT_MERGED[((CRSP_COMPUSTAT_MERGED['sic']<6000) |  (CRSP_COMPUSTAT_MERGED['sic']>=7000)) &
                                              (CRSP_COMPUSTAT_MERGED['exchg'].isin([11, 12, 13, 14, 15, 16, 17, 18, 19, 20]))                  
                                             ]
                   

In [52]:
# Split Up 'datacqtr' into calendar year and quarter
CRSP_COMPUSTAT_MERGED['CalendarYear'] = CRSP_COMPUSTAT_MERGED['datacqtr'].str.slice(0,4)
CRSP_COMPUSTAT_MERGED['Quarter'] = CRSP_COMPUSTAT_MERGED['datacqtr'].str.slice(4)

# Convert Quarters to Pandas Datetimes
CRSP_COMPUSTAT_MERGED['datacqtr_formatted'] = CRSP_COMPUSTAT_MERGED["CalendarYear"] + "-" + CRSP_COMPUSTAT_MERGED["Quarter"]
CRSP_COMPUSTAT_MERGED['QuarterStart'] = pd.PeriodIndex(CRSP_COMPUSTAT_MERGED['datacqtr_formatted'], freq='Q').to_timestamp()

# Calculate Start of Quarter and End of Quarter Dates
CRSP_COMPUSTAT_MERGED['QuarterEnd'] = CRSP_COMPUSTAT_MERGED['QuarterStart'] + pd.offsets.MonthEnd(3)
CRSP_COMPUSTAT_MERGED['QuarterStart_Month'] = CRSP_COMPUSTAT_MERGED['QuarterStart'].dt.to_period('m')
CRSP_COMPUSTAT_MERGED['QuarterEnd_Month'] = CRSP_COMPUSTAT_MERGED['QuarterEnd'].dt.to_period('m')

# Calculate Lagged Dates (Year-Month)
CRSP_COMPUSTAT_MERGED['Date_Lag2'] = CRSP_COMPUSTAT_MERGED['QuarterEnd_Month'] + 2
CRSP_COMPUSTAT_MERGED['Date_Lag3'] = CRSP_COMPUSTAT_MERGED['QuarterEnd_Month'] + 3
CRSP_COMPUSTAT_MERGED['Date_Lag4'] = CRSP_COMPUSTAT_MERGED['QuarterEnd_Month'] + 4
CRSP_COMPUSTAT_MERGED.head()

,GVKEY,LPERMNO,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,conm,curcdq,datacqtr,datafqtr,atq,ceqq,cheq,ltq,niq,exchg,costat,dlrsn,sic,dldte,CalendarYear,Quarter,datacqtr_formatted,QuarterStart,QuarterEnd,QuarterStart_Month,QuarterEnd_Month,Date_Lag2,Date_Lag3,Date_Lag4
0,1000,25881,12/31/1970,1970,$4.000,INDL,C,D,STD,A & E PLASTIK PAK INC,USD,1970Q4,1970Q4,$nan,$10.544,$nan,$nan,$-1.064,12,I,$9.000,3089,06/30/1978,1970,Q4,1970-Q4,1970-10-01,1970-12-31,1970-10,1970-12,1971-02,1971-03,1971-04
1,1000,25881,03/31/1971,1971,$1.000,INDL,C,D,STD,A & E PLASTIK PAK INC,USD,1971Q1,1971Q1,$nan,$nan,$nan,$nan,$0.346,12,I,$9.000,3089,06/30/1978,1971,Q1,1971-Q1,1971-01-01,1971-03-31,1971-01,1971-03,1971-05,1971-06,1971-07
2,1000,25881,06/30/1971,1971,$2.000,INDL,C,D,STD,A & E PLASTIK PAK INC,USD,1971Q2,1971Q2,$nan,$nan,$nan,$nan,$0.152,12,I,$9.000,3089,06/30/1978,1971,Q2,1971-Q2,1971-04-01,1971-06-30,1971-04,1971-06,1971-08,1971-09,1971-10
3,1000,25881,09/30/1971,1971,$3.000,INDL,C,D,STD,A & E PLASTIK PAK INC,USD,1971Q3,1971Q3,$nan,$nan,$nan,$nan,$-0.672,12,I,$9.000,3089,06/30/1978,1971,Q3,1971-Q3,1971-07-01,1971-09-30,1971-07,1971-09,1971-11,1971-12,1972-01
4,1000,25881,12/31/1971,1971,$4.000,INDL,C,D,STD,A & E PLASTIK PAK INC,USD,1971Q4,1971Q4,$29.330,$8.381,$2.557,$20.949,$-2.144,12,I,$9.000,3089,06/30/1978,1971,Q4,1971-Q4,1971-10-01,1971-12-31,1971-10,1971-12,1972-02,1972-03,1972-04


## CRSP (Monthly)

In [53]:
# Read in Dataframe
CRSP_MONTHLY = pd.read_csv('/content/drive/Shared drives/Financial Modeling and Testing/Project 1/Data/Original/CRSP_MONTHLY.csv')

# Convert to Datetime
CRSP_MONTHLY['date'] = pd.to_datetime(CRSP_MONTHLY['date'])

# Convert Date to Month Period
CRSP_MONTHLY['date_month'] = CRSP_MONTHLY['date'].dt.to_period('m')

# Filter Out 6000 Range SIC Companies (Financial and ETFs)
CRSP_MONTHLY['SICCD'] = pd.to_numeric(CRSP_MONTHLY['SICCD'], errors='coerce')
CRSP_MONTHLY = CRSP_MONTHLY[(CRSP_MONTHLY['SICCD']<6000) |  (CRSP_MONTHLY['SICCD']>=7000)]

# Filter Share Code to be 10 or 11
CRSP_MONTHLY['SHRCD'] = pd.to_numeric(CRSP_MONTHLY['SHRCD'], errors='coerce')
CRSP_MONTHLY = CRSP_MONTHLY[(CRSP_MONTHLY['SHRCD'].isin([10,11]))]

# Filter Share Class to be 'A' or NaN
CRSP_MONTHLY = CRSP_MONTHLY[((CRSP_MONTHLY['SHRCLS'].isna()) | (CRSP_MONTHLY['SHRCLS'] == 'A'))]

# Filter Out Returns less than -50
CRSP_MONTHLY['RET'] = pd.to_numeric(CRSP_MONTHLY['RET'], errors='coerce')
CRSP_MONTHLY = CRSP_MONTHLY[(CRSP_MONTHLY['RET']>-50)]

CRSP_MONTHLY.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PERMNO,date,SHRCD,SICCD,SHRCLS,PRC,RET,SHROUT,CFACPR,date_month
2,10000,1986-02-28,$10.000,"$3,990.000",A,$-3.250,$-0.257,"$3,680.000",$1.000,1986-02
3,10000,1986-03-31,$10.000,"$3,990.000",A,$-4.438,$0.365,"$3,680.000",$1.000,1986-03
4,10000,1986-04-30,$10.000,"$3,990.000",A,$-4.000,$-0.099,"$3,793.000",$1.000,1986-04
5,10000,1986-05-30,$10.000,"$3,990.000",A,$-3.109,$-0.223,"$3,793.000",$1.000,1986-05
6,10000,1986-06-30,$10.000,"$3,990.000",A,$-3.094,$-0.005,"$3,793.000",$1.000,1986-06


## CRSP (Daily)

In [54]:
# Load In Data
CRSP_DAILY = pd.read_csv('/content/drive/Shared drives/Financial Modeling and Testing/Project 1/Data/Original/CRSP_DAILY_REDUCED.csv')

In [55]:
# Change to numeric
CRSP_DAILY['RET'] = pd.to_numeric(CRSP_DAILY['RET'], errors='coerce')

## Sigma Calculation

In [56]:
window=90
CRSP_DAILY['SIGMA'] = ((((CRSP_DAILY['RET'].copy()**2).rolling(window).sum())/(window-1))*252)**(1/2)
CRSP_DAILY.head()

,Unnamed: 0,PERMNO,date,RET,SIGMA,date_month,lastDayOfMonth
0,19,10000,1986-01-31,$-0.014,$nan,1986-01,1986-01-31
1,38,10000,1986-02-28,$0.000,$nan,1986-02,1986-02-28
2,58,10000,1986-03-31,$0.007,$nan,1986-03,1986-03-31
3,80,10000,1986-04-30,$-0.015,$nan,1986-04,1986-04-30
4,101,10000,1986-05-30,$0.015,$nan,1986-05,1986-05-30


## Format Daily Data from Merge

In [57]:
# Convert to Datetime
CRSP_DAILY['date'] = pd.to_datetime(CRSP_DAILY['date'])

# Convert Date to Month Period
CRSP_DAILY['date_month'] = CRSP_DAILY['date'].dt.to_period('m')

# Keep Only Last Day of Month
lastDayOfMonthInfo = CRSP_DAILY.groupby(['PERMNO', 'date_month'])['date'].max().reset_index()
lastDayOfMonthInfo = lastDayOfMonthInfo.rename(columns={'date': 'lastDayOfMonth'})
temp = pd.merge(CRSP_DAILY,
                lastDayOfMonthInfo,
                left_on=['PERMNO', 'date_month'],
                right_on=['PERMNO', 'date_month']
)
CRSP_DAILY = temp.copy()
CRSP_DAILY = CRSP_DAILY[CRSP_DAILY['date'] == CRSP_DAILY['lastDayOfMonth']]   #3 For some reasons this block needed to be ran twice
CRSP_DAILY.head()

KeyError: ignored

## SP500 Data

In [ ]:
# Read in Data
SP500_MONTHLY = pd.read_csv('/content/drive/Shared drives/Financial Modeling and Testing/Project 1/Data/Original/SP500_MONTHLY.csv')

# Format caldt as datetime
SP500_MONTHLY['caldt'] = pd.to_datetime(SP500_MONTHLY['caldt'])

# Convert Date to Month Period
SP500_MONTHLY['date_month'] = SP500_MONTHLY['caldt'].dt.to_period('m')

# Adjust totval (quoted in $1000s)
SP500_MONTHLY['totval'] = SP500_MONTHLY['totval']*1000

# Drop caldt
SP500_MONTHLY = SP500_MONTHLY.drop(columns=['caldt'])

# Rename Columns (to make later merge simpler)
SP500_MONTHLY = SP500_MONTHLY.rename(columns={'vwretd': 'vwretdSP500', 'totval': 'totvalSP500'})

SP500_MONTHLY.head()

## Paper Data

In [ ]:
paperData = pd.read_csv('/content/drive/Shared drives/Financial Modeling and Testing/Project 1/Data/Original/paperData.csv')
paperData = paperData.set_index(['year'])

# Merge Dataframes

## Merge CRSP/COMPUSTAT Merged Data set with CRSP (Monthly)

In [ ]:
# Select Accounting Features to Merge into CRSP Monthly Dataframe
CRSP_COMPUSTAT_features = ['atq', 'ceqq', 'cheq', 'ltq', 'niq']

# Select Features to Keep after Merge
featuresToKeep = ['PERMNO', 'date_month', 'PRC', 'SHROUT', 'CFACPR']

# Add Accounting Features to Features to keep after merge
featuresToKeep.extend(CRSP_COMPUSTAT_features)

# Add Lagged Accounting Features
for lag in range(2,5):
    CRSP_COMPUSTAT_merge_features = ['LPERMNO', f'Date_Lag{lag}']
    CRSP_COMPUSTAT_merge_features.extend(CRSP_COMPUSTAT_features)
    if lag==2:
        temp = pd.merge(CRSP_COMPUSTAT_MERGED[CRSP_COMPUSTAT_merge_features],
                                 CRSP_MONTHLY[['PERMNO', 'date_month', 'PRC', 'SHROUT', 'CFACPR']],
                                 how='right',
                                 left_on=['LPERMNO', f'Date_Lag{lag}'],
                                 right_on=['PERMNO', 'date_month']
                                )
        temp = temp[featuresToKeep]
        
    else:
        temp = pd.merge(CRSP_COMPUSTAT_MERGED[CRSP_COMPUSTAT_merge_features],
                                 temp[featuresToKeep],
                                 how='right',
                                 left_on=['LPERMNO', f'Date_Lag{lag}'],
                                 right_on=['PERMNO', 'date_month'],
                        suffixes=('', '_y')
                                )
                      
        # Update Features
        for feature in CRSP_COMPUSTAT_features:
            temp[feature] = temp[feature].fillna(temp[f'{feature}_y'])
            temp = temp.drop([f'{feature}_y'], 1)
        
        temp = temp[featuresToKeep]

explanatoryDataFrame = temp.copy()        
explanatoryDataFrame[:10]

## Merge Existing Explanatory Data set with CRSP (Daily)

In [ ]:
explanatoryDataFrame = pd.merge(explanatoryDataFrame,
                CRSP_DAILY[['PERMNO', 'date_month', 'SIGMA']],
                how='left',
                left_on=['PERMNO', 'date_month'],
                right_on=['PERMNO', 'date_month']
               )
explanatoryDataFrame[:10]

## Merge Existing Explanatory Data Set with SP500 Monthly

In [ ]:
explanatoryDataFrame = pd.merge(explanatoryDataFrame,
                SP500_MONTHLY,
                how='left',
                left_on=['date_month'],
                right_on=['date_month']
               )
explanatoryDataFrame[:10]

In [ ]:
len(explanatoryDataFrame['PERMNO'].unique())

# Remaining Explanatory Variable Calculations

## NITA

In [ ]:
# Precomuputations
explanatoryDataFrame['ME'] = explanatoryDataFrame['PRC'] * explanatoryDataFrame['SHROUT']
explanatoryDataFrame['BE'] = explanatoryDataFrame['ceqq']
explanatoryDataFrame['TA'] = explanatoryDataFrame['atq']

# Total Assets Adjusted Calculation
explanatoryDataFrame['totalAssetsAdj'] = explanatoryDataFrame['TA'] + 0.1*(explanatoryDataFrame['ME'] - explanatoryDataFrame['BE'])

# NITA Calculation
explanatoryDataFrame['NITA'] = explanatoryDataFrame['niq'] / explanatoryDataFrame['totalAssetsAdj']

## NIMTA

In [ ]:
# NIMTA Calculation
explanatoryDataFrame['NIMTA'] = explanatoryDataFrame['niq'] / (explanatoryDataFrame['ME'] + explanatoryDataFrame['ltq'])

## TLTA

In [ ]:
explanatoryDataFrame['TLTA'] = explanatoryDataFrame['ltq'] / explanatoryDataFrame['totalAssetsAdj']

## TLMTA

In [ ]:
explanatoryDataFrame['TLMTA'] = explanatoryDataFrame['ltq'] / (explanatoryDataFrame['ME'] + explanatoryDataFrame['ltq'])

## EXRET

In [ ]:
explanatoryDataFrame['adjPRC'] = explanatoryDataFrame['PRC'] * explanatoryDataFrame['CFACPR']
explanatoryDataFrame['RET'] = explanatoryDataFrame['adjPRC'].shift(1) / explanatoryDataFrame['adjPRC'] - 1
explanatoryDataFrame['EXRET'] = np.log(1+explanatoryDataFrame['RET']) - np.log(1+explanatoryDataFrame['vwretdSP500'])

# Drop -inf values for EXRET

## RSIZE

In [ ]:
explanatoryDataFrame['RSIZE'] = np.log(explanatoryDataFrame['ME'] / explanatoryDataFrame['totvalSP500'])

## CASHMTA

In [ ]:
explanatoryDataFrame['CASHMTA'] = explanatoryDataFrame['cheq'] / (explanatoryDataFrame['ME'] + explanatoryDataFrame['ltq'])

In [ ]:
explanatoryDataFrame.head()

### Plot Number of Firms

In [ ]:
test = explanatoryDataFrame.dropna()
test.groupby(['date_month'])['PERMNO'].count().plot()

# Bankruptcy Indicator

## DLRSN

In [ ]:
DLRSN = CRSP_COMPUSTAT_MERGED[['LPERMNO', 'dlrsn', 'dldte']]

DLRSN = DLRSN.drop_duplicates()
bankruptcies_DLRSN = DLRSN.copy()[DLRSN['dlrsn']==2.0]

# Convert to Datetime
bankruptcies_DLRSN['dldte'] = pd.to_datetime(bankruptcies_DLRSN['dldte'])

# Convert Date to Month Period
bankruptcies_DLRSN['dldte_year'] = bankruptcies_DLRSN['dldte'].dt.to_period('y')

bankruptcies_DLRSN = bankruptcies_DLRSN.groupby(['dldte_year'])['LPERMNO'].count()
bankruptcies_DLRSN = pd.DataFrame(bankruptcies_DLRSN)
bankruptcies_DLRSN.index = bankruptcies_DLRSN.index.rename('year')
bankruptcies_DLRSN = bankruptcies_DLRSN.rename(columns={'LPERMNO': 'Bankruptcies (DLRSN)'})
bankruptcies_DLRSN.index = bankruptcies_DLRSN.index.astype(str).astype(int)

bankruptcies_DLRSN.head()

## DLSTCD

In [ ]:
DLSTCD = pd.read_csv('/content/drive/Shared drives/Financial Modeling and Testing/Project 1/Data/Original/output.csv')

# Format caldt as datetime
DLSTCD['date'] = pd.to_datetime(DLSTCD['date'], format='%Y%m%d')

# Convert Date to Month Period
DLSTCD['date_month'] = DLSTCD['date'].dt.to_period('m')

# Convert Date to Year Period
DLSTCD['date_year'] = DLSTCD['date'].dt.to_period('y')

validPERMNOS = explanatoryDataFrame['PERMNO'].unique()
DLSTCD = DLSTCD[DLSTCD['PERMNO'].isin(validPERMNOS)]
DLSTCD = DLSTCD[DLSTCD['isBankrupt']==1]
bankruptcies_DLSTCD = DLSTCD.groupby(['date_year'])['PERMNO'].count()
bankruptcies_DLSTCD = pd.DataFrame(bankruptcies_DLSTCD)
bankruptcies_DLSTCD.index = bankruptcies_DLSTCD.index.rename('year')
bankruptcies_DLSTCD = bankruptcies_DLSTCD.rename(columns={'PERMNO': 'Bankruptcies (DLSTCD)'})
bankruptcies_DLSTCD.index = bankruptcies_DLSTCD.index.astype(str).astype(int)

bankruptcies_DLSTCD.head()

# Plotting

## Active Firms

In [ ]:
temp = explanatoryDataFrame.copy()

# Count number of Firms per month
activeFirmsAnalysis = explanatoryDataFrame.groupby(['date_month'])['PERMNO'].count()

# Convert Series to Dataframe
activeFirmsAnalysis = pd.DataFrame(activeFirmsAnalysis)

# Add Year Column
activeFirmsAnalysis['year'] = activeFirmsAnalysis.index.to_timestamp().to_period('y')

# Keep only last count of companies for month per year
activeFirmsAnalysis = activeFirmsAnalysis.reset_index()
activeFirmsAnalysis = activeFirmsAnalysis.drop_duplicates(subset=['year'], keep='last')
activeFirmsAnalysis = activeFirmsAnalysis.groupby(['year'])['PERMNO'].mean()

# Convert to DataFrame
activeFirmsAnalysis = pd.DataFrame(activeFirmsAnalysis)

# Rename and Reformat Columns/Index before merge
activeFirmsAnalysis = activeFirmsAnalysis.rename(columns={'PERMNO': 'Active Firms (Calculated)'})
activeFirmsAnalysis.index = activeFirmsAnalysis.index.astype(str).astype(int)
activeFirmsAnalysis = activeFirmsAnalysis.join(paperData['activeFirms'])
activeFirmsAnalysis = activeFirmsAnalysis.rename(columns={'activeFirms': 'Active Firms (Paper)'})

# Plot
activeFirmsAnalysis_Plot = activeFirmsAnalysis.plot(title='Created Universe vs Paper Universe',
                                                    ylabel='Number of Active Firms'
                                                   
                                                   )
fig = activeFirmsAnalysis_Plot.get_figure()
fig.savefig('activeFirmsComparison.png')

In [ ]:
# Join Dataframes
bankruptciesAnalysis = bankruptcies_DLSTCD.join(bankruptcies_DLRSN)
bankruptciesAnalysis = bankruptciesAnalysis.join(paperData['bankruptcies'])

# Replace NaN Values that are supposed to be 0 (no bankrupcties occured)
bankruptciesAnalysis['Bankruptcies (DLRSN)'][5:] = bankruptciesAnalysis['Bankruptcies (DLRSN)'][5:].fillna(0)

# Rename Column
bankruptciesAnalysis = bankruptciesAnalysis.rename(columns={'bankruptcies': 'Bankruptcies (Paper)'})


# Plot
bankruptciesAnalysis_Plot = bankruptciesAnalysis.plot(title='Created Bankruptcy Indicators vs Paper Indicator',
                                                      ylabel='Number of Bankruptcies'
                                                     )
# Save Figure
fig = bankruptciesAnalysis_Plot.get_figure()
fig.savefig('bankruptcyComparison.png')